In [1]:
import sys
import os
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
print(parent_dir)
project_root = parent_dir
import polyscope.imgui as psim
import scipy as sp
import numpy as np
import jax
import jax.numpy as jnp
import jax.random as jrandom
import jax.nn as nn
import jax.lax as lax
import jax.random as jrandom
import jax.nn as nn
import jax.lax as lax
import os
import jax.experimental.sparse as jsp
from scipy.sparse import diags, csr_matrix
from src.dataGenerator.spherical_data_generator import *
from src.utils.sht_helper import *
from src.stochastics.sde import *
from src.stochastics.sde_solver import *
import matplotlib.pyplot as plt
import open3d as o3d


import random
def get_random_int():
    return random.randint(0, 1000000)


/home/zjc/PycharmProjects/ShapeSphericalFNO/ShapeSphericalFNO
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
import s2fft
# import s2wav
# from s2ball.transform import *
from src.utils.sht_helper import *

L = 20
n_side = 5
# print(np.arange(12 * n_side **2 ))
data_generator = S2ManifoldDataGenerator(sampling="healpix", manifold_type='real_data', file_path=project_root + "/data/morph/Sylvilagus_aquaticus_132343_endo-000417436.stl", scale=1.0, src_type='mesh')
# data_generator = S2ManifoldDataGenerator(sampling="mw", manifold_type='real_data', file_path=project_root + "/data/morph/Sylvilagus_aquaticus_132343_endo-000417436.stl", scale=1.0, src_type='mesh')
# data = data_generator.generate_data(L, n_side, 1, healpix=True)
data = data_generator.generate_data(L, nside=n_side, healpix=True)

data = data[0]
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[:,0], data[:,1], data[:,2])
plt.show()
# data_generator.sampling = "healpix"
# method = "jax_healpy"

print(data.shape)
# data_sht = jax.vmap(s2fft.forward, in_axes=(2, None))(data, data_generator.L, sampling=data_generator.sampling, method="jax")
# data_sht = []
# for i in range(data.shape[2]):
#     data_sht.append(s2fft.forward(data[:,:,i], data_generator.L, sampling=data_generator.sampling, method="jax"))
# data_sht = jnp.stack(data_sht, axis=-1)
# L = 15
# x = np.array(data)
data_rfft = jax.vmap(lambda x: jnp.fft.rfft2(x, ( 3*L,3*L), axes=(0,1)), in_axes=(2))(data)
data_rfft = jnp.transpose(data_rfft, (1, 2, 0))
data_rfft_inv = jax.vmap(lambda x: jnp.fft.irfft2(x, ( 3*L,2*L), axes=(0,1)), in_axes=(2))(data_rfft)
data_rfft_inv = jnp.transpose(data_rfft_inv, (1, 2, 0))
print(data_rfft_inv.shape)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_rfft_inv[:,:,0], data_rfft_inv[:,:,1], data_rfft_inv[:,:,2])
plt.show()

data_sht = jax.vmap(lambda x: s2fft.forward(x, L = L, reality=True, sampling=data_generator.sampling, method="jax"), in_axes=(2))(data)
# data_sht = jax.vmap(lambda x: harmonic.forward(x, L = L), in_axes=(2))(data)
# data = data.transpose(2, 0, 1)
# data_sht = s2fft.forward(data, L = L, reality=True, sampling=data_generator.sampling, method="jax", spin=1)
data_sht = jnp.transpose(data_sht, (1, 2, 0))
print("data_sht.shape: ", data_sht.shape)

# data_sht = resize_flm(data_sht, 5 * L)
inv_data_sht = jax.vmap(lambda x: s2fft.inverse(x, L = L, reality=True, sampling=data_generator.sampling, method="jax"), in_axes=(2))(data_sht)
# inv_data_sht = jax.vmap(lambda x: harmonic.inverse(x, L = 5 * L), in_axes=(2))(data_sht)
inv_data_sht = jnp.transpose(inv_data_sht, (1, 2, 0))
print(inv_data_sht[1,:,:])
inv_data_real = jnp.real(inv_data_sht)
print(inv_data_real.shape)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(inv_data_real[:,:,0], inv_data_real[:,:,1], inv_data_real[:,:,2])
plt.show()